In [1]:
import sys
sys.path.append("/workspaces/mlp-playground/")

In [2]:
import flaxmodels

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from playground.sim_clr import model

In [5]:
?flaxmodels.ResNet18

Signature:
flaxmodels.ResNet18(
    output='softmax',
    pretrained='imagenet',
    normalize=True,
    num_classes=1000,
    kernel_init=<function variance_scaling.<locals>.init at 0x7f92a25e6d40>,
    bias_init=<function zeros at 0x7f92a3536680>,
    ckpt_dir=None,
    dtype='float32',
)
Docstring:
Implementation of the ResNet18 by He et al.
Reference: https://arxiv.org/abs/1512.03385

The pretrained parameters are taken from:
https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

Args:
    output (str):
        Output of the module. Available options are:
            - 'softmax': Output is a softmax tensor of shape [N, 1000] 
            - 'log_softmax': Output is a softmax tensor of shape [N, 1000] 
            - 'logits': Output is a tensor of shape [N, 1000]
            - 'activations': Output is a dictionary containing the ResNet activations
    pretrained (str):
        Indicates if and what type of weights to load. Options are:
            - 'imagenet': L